Reference: https://www.kaggle.com/reynaldo/naive-xgb/comments/notebook

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import model_selection, preprocessing
from xgboost.sklearn import XGBRegressor
import datetime
import math
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

pd.options.display.max_columns = 300
%matplotlib inline  

/home/tseidakhmetov/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_train = pd.read_csv('train.csv')#, parse_dates=['timestamp'])
df_test = pd.read_csv('test.csv')#, parse_dates=['timestamp'])
df_macro = pd.read_csv('macro.csv')#, parse_dates=['timestamp'])
id_test = df_test.id
df_train.sample(3)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_av

In [3]:
trainsub = df_train[df_train.timestamp < '2015-01-01']
trainsub = trainsub[trainsub.product_type=="Investment"]

ind_1m = trainsub[trainsub.price_doc <= 1000000].index
ind_2m = trainsub[trainsub.price_doc == 2000000].index
ind_3m = trainsub[trainsub.price_doc == 3000000].index

train_index = set(df_train.index.copy())

for ind, gap in zip([ind_1m, ind_2m, ind_3m], [10, 3, 2]):
    ind_set = set(ind)
    ind_set_cut = ind.difference(set(ind[::gap]))

    train_index = train_index.difference(ind_set_cut)

df_train = df_train.loc[train_index]

In [4]:
y_train = df_train["price_doc"]
x_train = df_train.drop(["id", "timestamp", "price_doc"], axis=1)
x_test = df_test.drop(["id", "timestamp"], axis=1)

#can't merge train with test because the kernel run for very long time

for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values)) 
        x_train[c] = lbl.transform(list(x_train[c].values))
        #x_train.drop(c,axis=1,inplace=True)
        
for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[c].values)) 
        x_test[c] = lbl.transform(list(x_test[c].values))
        #x_test.drop(c,axis=1,inplace=True)   

In [5]:
# xreg = XGBRegressor(learning_rate=0.05,
#                     max_depth = 5,
#                     subsample = 0.7,
#                     colsample_bytree = 0.7,
#                     objective = 'reg:linear',
#                     nthread = 3,
#                     silent= 1)  # and for classifier  
# xreg.fit(x_train, y_train)  
import scipy.stats as st

one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

params = {  
    "n_estimators": st.randint(3, 40),
    "max_depth": st.randint(3, 40),
    "learning_rate": st.uniform(0.05, 0.4),
    "colsample_bytree": one_to_left,
    "subsample": one_to_left,
    "gamma": st.uniform(0, 10),
    'reg_alpha': from_zero_positive,
    "min_child_weight": from_zero_positive,
}
iters = 100

xgbreg = XGBRegressor()

In [6]:
def rmsle_XGB(labels, preds):
    labels = labels.tolist()
    preds = preds.tolist()
#     print len(labels), len(preds)
    terms_to_sum = [(math.log(labels[i] + 1) - math.log(max(0, preds[i]) + 1)) ** 2.0 for i, pred in enumerate(labels)]
    return (sum(terms_to_sum) * (1.0 / len(preds))) ** 0.5

rmsle_scorer = make_scorer(rmsle_XGB, greater_is_better=False)#, needs_proba=True)

In [7]:
gs = RandomizedSearchCV(xgbreg, params, n_iter=iters, n_jobs=-1, scoring=rmsle_scorer)
gs.fit(x_train, y_train)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
          fit_params={}, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7f39aaf510>, 'subsample': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7f9c574590>, 'reg_alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7f9c5747d0>, 'colsample_bytree': <...at 0x7f7f39ac4b10>, 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7f39ac4150>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True,
          scoring=make_scorer(

In [ ]:
gs.cv_results_

In [8]:
gs.best_score_

-0.33697058948390857

In [9]:
gs.best_estimator_
# XGBRegressor(base_score=0.5, colsample_bylevel=1,
#        colsample_bytree=0.90376743790350766, gamma=0.82229270633305096,
#        learning_rate=0.1059352041807113, max_delta_step=0, max_depth=39,
#        min_child_weight=34.807721810404857, missing=None, n_estimators=30,
#        nthread=-1, objective='reg:linear', reg_alpha=25.265495286611461,
#        reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
#        subsample=0.85828449473027224)

XGBRegressor(base_score=0.5, colsample_bylevel=1,
       colsample_bytree=0.90376743790350766, gamma=0.82229270633305096,
       learning_rate=0.1059352041807113, max_delta_step=0, max_depth=39,
       min_child_weight=34.807721810404857, missing=None, n_estimators=30,
       nthread=-1, objective='reg:linear', reg_alpha=25.265495286611461,
       reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.85828449473027224)

In [10]:
model = gs.best_estimator_.fit(x_train, y_train)

In [11]:
preds = model.predict(x_train)

In [12]:
rmsle_XGB(preds, y_train)

0.2549558663592191

In [7]:
# xgb_params = {
#     'eta': 0.05,
#     'max_depth': 5,
#     'subsample': 0.7,
#     'colsample_bytree': 0.7,
#     'objective': 'reg:linear',
#     'eval_metric': 'rmse',
#     'silent': 1
# }

# dtrain = xgboost.DMatrix(x_train, y_train)
# dtest = xgb.DMatrix(x_test)

In [13]:
y_predict = model.predict(x_test)
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
output.head()

,id,price_doc
0,30474,5181986.0
1,30475,8368197.5
2,30476,5626213.5
3,30477,5818185.5
4,30478,4847722.0


In [14]:
output.shape

(7662, 2)

In [15]:
output.to_csv('submissions/xgb_sklearn.csv', index=False)